In [1]:
import pandas as pd
import numpy as np

In [2]:
N = 1
T1 = 4
T2 = 10
T3 = 10
PROC_NUM = 3
QUEUE_SIZE = 16
MAX_STATE = PROC_NUM + QUEUE_SIZE
MU = 1 / T1
NU = 1 / T2

In [3]:
def get_lambda(i):
    return N

In [4]:
def get_mu(i):
    if i <= PROC_NUM:
        return i * MU
    else:
        return PROC_NUM * MU + (i - PROC_NUM) * NU

In [5]:
def state_prob(i):
    prob = 1;
    if i == 0:
        for i in range(1, MAX_STATE + 1):
            prob += state_prob(i)
        prob = 1 / prob
    else:
        prob = 1
        for i in range(1, i + 1):
            prob *= get_lambda(i) / get_mu(i)
    return prob

In [6]:
df = pd.DataFrame(index=np.arange(MAX_STATE + 1), columns=('prob','busy','queue'))
df.index.name = 'state'

In [7]:
sum = 0
for i in range(0, MAX_STATE + 1):
    if i == 0:
        df.prob[i] = state_prob(0)
        sum += df.prob[i]
    else:
        df.prob[i] = df.prob[0] * state_prob(i)
        sum += df.prob[i]
df.prob.sum()

1.0

Среднее количество занятых работников рассчитаем сумму произведений вероятностей состояний на количество занятых работников в них:

In [8]:
def busy_count(i):
    return i if i <= PROC_NUM else PROC_NUM

In [9]:
for i in range(0, MAX_STATE + 1):
    df.busy[i] = busy_count(i)
print('{} = {:.5f}'.format('busy', df.busy.multiply(df.prob).sum()))

busy = 2.80873


Средняя длина очереди считается аналогично: сумма произведений вероятностей состояний на длину очереди в них:

In [10]:
def queue_length(i):
    return 0 if i <= PROC_NUM else i - PROC_NUM

In [11]:
for i in range(0, MAX_STATE + 1):
    df.queue[i] = queue_length(i)
print('{} = {:.5f}'.format('queue', df.queue.multiply(df.prob).sum()))

queue = 2.97698


In [12]:
df.to_csv('../data/data.csv')
df.append(df.sum(), ignore_index=True)

,prob,busy,queue
0,0.0100668,0,0
1,0.0402672,1,0
2,0.0805345,2,0
3,0.107379,3,0
4,0.126329,3,1
5,0.132977,3,2
6,0.126645,3,3
7,0.110126,3,4
8,0.088101,3,5
9,0.06526,3,6


Вероятность отказа из-за заполненной очереди равна вероятности последнего состояния:

In [13]:
reject_prob = df.prob[MAX_STATE]
print('{} = {:.5f}'.format('reject_prob', reject_prob))

reject_prob = 0.00012


Вероятность ухода из очереди равна отношению интенсивности ухода к интенсивности поступления заявок, умноженному на среднюю длину очереди:

In [18]:
leave_prob = NU / N * df.queue.multiply(df.prob).sum()
print('{} = {:.5f}'.format('leave_prob', leave_prob))

leave_prob = 0.29770


Вероятность успешного обслуживания найдем как произведение вероятностей отстутствия отказа обслуживания и ухода из очереди:

In [19]:
success_prob = (1 - leave_prob) * (1 - reject_prob)
print('{} = {:.5f}'.format('success_prob', success_prob))

success_prob = 0.70222


Среднее количество человек в системе найдем как сумму средней длины очереди и среднего количества занятых каналов:

In [22]:
mean_cnt = df.queue.multiply(df.prob).sum() + df.busy.multiply(df.prob).sum()
print('{} = {:.5f}'.format('mean_cnt', mean_cnt))

mean_cnt = 5.78571


Среднее время нахождения в системе найдем с помощью закону Литтла: $t_c = \dfrac{j}{\lambda} + T$, где $T$ - время обеда, умноженное на вероятность успешного обслуживания:

In [23]:
mean_time = mean_cnt / N + T3 * success_prob
print('{} = {:.5f}'.format('mean_time', mean_time))

mean_time = 12.80789
